In [52]:
import csv
import numpy as np

In [53]:
tweets = []
vocabulary = set()

with open("Preprocessed.csv", encoding="utf8") as file:
    data = csv.reader(file)
    data_list = list(data)

    for tweet in data_list:

        sentence = tweet[2]
        tweets.append(sentence)

print(len(tweets))


4287


In [54]:
bigram_list = []
bigram_count = {}
word_count = {}

for tweet in tweets:

    tokenized_tweet = tweet.split()

    for i in range(len(tokenized_tweet)):
        
        if(i<len(tokenized_tweet)-1):
            bigram = (tokenized_tweet[i], tokenized_tweet[i+1])
            bigram_list.append(bigram)

            if bigram in bigram_count:
                bigram_count[bigram] += 1
            else:
                bigram_count[bigram] = 1
        
        if tokenized_tweet[i] in word_count:
            word_count[tokenized_tweet[i]] += 1
        else:
            word_count[tokenized_tweet[i]] = 1
    
        vocabulary.add(tokenized_tweet[i])
        # vocabulary.add(tokenized_tweet[i+1])
        
print(len(bigram_list))
print(len(bigram_count))
print(len(word_count))
print(len(vocabulary))

24848
22069
5736
5736


In [55]:
unigram_probability_matrix = {}
total_words_in_vocab = len(vocabulary)

for i in word_count:
    unigram_probability_matrix[i] = word_count[i]/total_words_in_vocab


In [56]:
bigram_probability_matrix = np.zeros((len(vocabulary), len(vocabulary)))

vocabulary_list = list(vocabulary)

for row in range(len(vocabulary_list)):
    for column in range(len(vocabulary_list)):

        current_bigram = (vocabulary_list[row], vocabulary_list[column])
        current_bigram_count = 0

        if current_bigram in bigram_count:
            current_bigram_count = bigram_count[current_bigram]
            
        bigram_probability_matrix[row, column] = (current_bigram_count + 1) / (word_count[vocabulary_list[row]] + len(vocabulary))

In [57]:
positive_unigram = {}
negative_unigram = {}

for i in vocabulary_list:
    positive_unigram[i] = 1
    negative_unigram[i] = 1


positive_words = 0
negative_words = 0
# print(data_list)
for i in data_list:
    words = i[2].split()
    # print(words)
    if i[0] == '0':
        for k in words:
            # print(k)
            negative_unigram[k] +=1
            negative_words+=1
    elif i[0] == '1':
        for k in words:
            positive_unigram[k] +=1
            positive_words +=1

# print(negative_unigram)

for i in positive_unigram:
    positive_unigram[i] = positive_unigram[i] / (positive_words + len(vocabulary_list))
    
for i in negative_unigram:
    negative_unigram[i] = negative_unigram[i] / (negative_words + len(vocabulary_list))
    
ratio_unigram = {}

for i in positive_unigram:
    ratio_unigram[i] = positive_unigram[i]/negative_unigram[i] -1
    # ratio_unigram[i] = abs(math.log10(ratio_unigram[i]))
    # ratio_unigram[i] = (10 ** ratio_unigram[i]) -1 
    
# print(ratio_unigram)

In [58]:
# with np.printoptions(threshold=np.inf):
# np.savetxt("Matrix.csv", bigram_probability_matrix, delimiter =",", fmt ='% s')
# print(bigram_probability_matrix)

In [59]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math
sentiment_analyzer_object = SentimentIntensityAnalyzer()

# vocab_sentiment = {}
# bigram_sentiment = np.zeros((len(vocabulary), len(vocabulary)))

# # vocabulary_list = list(vocabulary)
# # print(vocabulary_list)

# for row in range(len(vocabulary_list)):
#     for column in range(len(vocabulary_list)):
#         bigram = vocabulary_list[row] + " " + vocabulary_list[column]
#         sentiment = sentiment_analyzer_object.polarity_scores(bigram)
#         bigram_sentiment[row,column] = 4 + (math.log10(bigram_probability_matrix[row, column])) + abs((sentiment['compound']))*10
        


     
# np.savetxt("Matrix.csv", bigram_sentiment, delimiter =",", fmt ='% s')

In [60]:
# unigram_sentiment = {}

# # vocabulary_list = list(vocabulary)
# # print(vocabulary_list)

# for i in range(len(vocabulary_list)):
#      sentiment = sentiment_analyzer_object.polarity_scores(vocabulary_list[i])
#      unigram_sentiment[vocabulary_list[i]] = 4 + (math.log10(unigram_probability_matrix[vocabulary_list[i]])) + abs((sentiment['compound'])) 
     
# np.savetxt("UniMatrix.csv", unigram_sentiment, delimiter =",", fmt ='% s')
# print(unigram_sentiment)

In [334]:
import random

random_sentence_list = []
num = 0
while (len(random_sentence_list) != 500):
    
    random_word_list = []
    rand = random.randrange(8,9)
    for i in range(rand):
        random_word_list.append(random.randrange(0, len(vocabulary_list)))
    ratio_score = 0   
    bigram_score = 1
    sentiment_score =0
    for i in range(len(random_word_list)):
        ratio_score+= ratio_unigram[vocabulary_list[random_word_list[i]]]
        sentiment = sentiment_analyzer_object.polarity_scores(vocabulary_list[random_word_list[i]])
        sentiment_score += sentiment['compound']*10

    # print(ratio_score)
    for i in range(len(random_word_list)-1):
        if(i==0):
            bigram_score += 4+ math.log10(unigram_probability_matrix[vocabulary_list[random_word_list[i]]])
            
        bigram_score += 4+ math.log10(bigram_probability_matrix[random_word_list[i], random_word_list[i+1]])
    # print(bigram_score)    
    if (abs(bigram_score) > 1 and (ratio_score>1 or ratio_score <-0.1) and sentiment_score!=0 ):
        final_sentence = []
        
        for x in random_word_list:
            final_sentence.append(vocabulary_list[x])
        
        sentence = " ".join(final_sentence)
        sentiment = sentiment_analyzer_object.polarity_scores(sentence)
        random_sentence_list.append(sentence)
        
# print(num)        
# print(random_sentence_list)


In [335]:
generated_samples = []

for i in random_sentence_list:
    sentiment = sentiment_analyzer_object.polarity_scores(i)
    if(sentiment['compound'] > 0):
        generated_samples.append(['1',i])
    elif sentiment['compound']<0:
        generated_samples.append(['0',i])
        
        
# print(generated_samples)

In [336]:
def perplexity_calculator(sentence):

    final_perplexity = 1

    tokenized_sentence = sentence.split()
    sentence_length = len(tokenized_sentence)

    first_word_base = unigram_probability_matrix[tokenized_sentence[0]]
    first_word_power = -(1/sentence_length)


    final_perplexity *= pow(first_word_base, first_word_power)

    for i in range(sentence_length-1):

        word1 = tokenized_sentence[i]
        word2 = tokenized_sentence[i+1]

        temp_base = (bigram_probability_matrix[(vocabulary_list.index(word1), vocabulary_list.index(word2))])
        temp_power = -(1/sentence_length)

        final_perplexity *= pow(temp_base, temp_power)


    return final_perplexity

perplexity_calculator("good morning world another mother")

768.4063039900251

In [337]:
total_generated_perplexity = 0

for i in random_sentence_list:
    total_generated_perplexity += perplexity_calculator(i)

avg_generated_perplexity = total_generated_perplexity/len(random_sentence_list)
print(avg_generated_perplexity)

5214.258552466331


In [338]:
import sklearn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

def train_and_evaluate(train_sentences, train_labels,test_sentences, test_labels):
    '''
    parameters:
    train_sentences : list of training sentences
    train_labels : list of training labels
    test_sentences : list of test sentences
    test_labels : list of test labels
    output:
    accuracy : accuracy of the test set
    '''
    # Model building
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    # Training the model with the training data
    model.fit(train_sentences, train_labels)
    # Predicting the test data categories
    predicted_test_labels = model.predict(test_sentences)
    return accuracy_score(test_labels, predicted_test_labels)

## DESIRED OUTCOMES

### PART A

In [339]:
# PART a is bigram_probability_matrix

In [340]:
# PART b
# Top 4 bigrams and their scores after smoothing

duplicate_bigram_probaility_matrix = bigram_probability_matrix.copy()

index = np.argpartition(-duplicate_bigram_probaility_matrix.ravel(), 4)[:4]
top_4 = np.column_stack(np.unravel_index(index, duplicate_bigram_probaility_matrix.shape))


print("Top 4 bigrams and their scores: ")

for i in top_4:
    print(f"Bigram = {vocabulary_list[i[0]], vocabulary_list[i[1]]} | Score after smoothing= {bigram_probability_matrix[i[0], i[1]]}")

Top 4 bigrams and their scores: 
Bigram = ('cant', 'wait') | Score after smoothing= 0.005936227951153324
Bigram = ('good', 'morning') | Score after smoothing= 0.007133377571333776
Bigram = ('last', 'night') | Score after smoothing= 0.005311857436600411
Bigram = ('look', 'like') | Score after smoothing= 0.004793699708953946


In [341]:
# PART c

sentences_A = []
label_A =[]

with open("Preprocessed.csv", encoding="utf8") as file:
    data = csv.reader(file)
    data_list = list(data)

    for tweet in data_list:
        label_A.append(tweet[0])
        sentences_A.append(tweet[2])
        
# print(label_A)     
sentences_Test = []
label_Test =[]

with open("Preprocessed_test.csv", encoding="utf8") as file:
    data = csv.reader(file)
    data_list = list(data)

    for tweet in data_list:
        label_Test.append(tweet[0])
        sentences_Test.append(tweet[2])

result_A = train_and_evaluate(sentences_A,label_A,sentences_Test,label_Test)

print(result_A)




0.8835403726708074


### PART B

In [342]:
#Part b

# np.savetxt("generated_samples.csv",generated_samples,delimiter=",",fmt="% s")

In [343]:
#Part c

# perpexility of generated 500 sentences
generated_sentences = []
generated_labels = []
generated_samples = []
with open("generated_samples.csv", encoding="utf8") as file:
    data = csv.reader(file)
    data_list = list(data)
    for sample in data_list:
        generated_samples.append(sample)
        generated_labels.append(sample[0])
        generated_sentences.append(sample[1])
    

total_generated_perplexity = 0

for i in generated_sentences:
    total_generated_perplexity += perplexity_calculator(i)

avg_generated_perplexity = total_generated_perplexity/len(random_sentence_list)
print(avg_generated_perplexity)

5183.7210848683135


In [344]:
#Part d

generated_positives = []
generated_negatives = []

for i in generated_samples:
    if i[0] == '1':
        generated_positives.append(i[1])
    
    if len(generated_positives) == 5 : break

for i in generated_samples:
    if i[0] == '0':
        generated_negatives.append(i[1])

    if len(generated_negatives) == 5 : break

print("Positives: ")
print(generated_positives)

print("Negatives: ")
print(generated_negatives)

Positives: 
['funky knob nyanza haircut disappear reef journey enjoyer', 'goodness tooth nye gnarly graphic bum koklass auction', 'guru entertainer manage mistake bunch kitty matta pout', 'vitamin surprise drag juke heaven moss mell abaze', 'harder beng ghoster waver inlayer thus sadly enjoy']
Negatives: 
['wolverine wese miserable mocha isogonal sumpit alma trade', 'lues busy single ghat atma deemer ugh development', 'battle graben uily puff otherwise trip hood imagine', 'waist must woman rapper amount reality ridiculously democrat', 'restless train dentist luke howso octoon city ladykin']


In [345]:
#Part e

sentences_B = []
label_B = []
sentences_B.extend(sentences_A)
label_B.extend(label_A)


label_B.extend(generated_labels)
sentences_B.extend(generated_sentences)


# for i in range(len(generated_samples)):
#     label_B.append(generated_samples[i][0])
#     sentences_B.append(generated_samples[i][1])
    
# print(sentences_B)
# print(label_B)
    
result_B = train_and_evaluate(sentences_B,label_B, sentences_Test, label_Test)

print(result_B)

0.8913043478260869
